# Estruturas Cripográficas - Criptografia e Segurança da Informação

## TP2 - Exercício 1

#### Enunciado

Estes problemas destinam à iniciação do uso do SageMath  em protótipos de esquemas clássicos de chave pública.

1. Construir uma classe Python que implemente o  EdDSA a partir do [“standard” FIPS186-5](https://csrc.nist.gov/publications/detail/fips/186/5/draft)
    1. A implementação deve conter funções para assinar digitalmente e verificar a assinatura.
2. A implementação da classe deve usar  uma das “Twisted Edwards Curves” definidas no standard e escolhida  na iniciação da classe= a curva  “edwards25519” ou “edwards448”.

# Versão do SageMath 

`SageMath 9.5`

 (Confirmar não se vou mudar)

METER PASSOS PARA INSTALAR + SECCÇÃO COM IMPORTS EM BAIXO


In [100]:
from sage.all import *
import os
import hashlib
import re 

#### Classe EdDSA - Capítulo 7 do FIPS186-5

No capitulo 7 tem de ter o input

# Meter aqui fonte !!!

In [101]:

class edPoint(object):
    def __init__(self,pt=None,curve=None,x=None,y=None):
        if pt != None:
            self.curve = pt.curve
            self.x = pt.x ; self.y = pt.y ; self.w = pt.w
        else:
            assert isinstance(curve,edCurve) #and curve.is_edwards(x,y)
            self.curve = curve
            self.x = x ; self.y = y ; self.w = x*y
    
    def eq(self,other):
        return self.x == other.x and self.y == other.y
    
    def copy(self):
        return edPoint(curve=self.curve, x=self.x, y=self.y)
    
    def zero(self):
        return edPoint(curve=self.curve,x=0,y=1)
    
    def sim(self):
        return edPoint(curve=self.curve, x= -self.x, y= self.y)
    
    def soma(self, other):
        a = self.curve.constants['a']; d = self.curve.constants['d']
        delta = d*self.w*other.w
        self.x, self.y  = (self.x*other.y + self.y*other.x)/(1+delta), (self.y*other.y - a*self.x*other.x)/(1-delta)
        self.w = self.x*self.y
        
    def duplica(self):
        a = self.curve.constants['a']; d = self.curve.constants['d']
        delta = d*(self.w)**2
        self.x, self.y = (2*self.w)/(1+delta) , (self.y**2 - a*self.x**2)/(1 - delta)
        self.w = self.x*self.y
        
    def mult(self, n):
        m = Mod(n,self.curve.L).lift().digits(2)   ## obter a representação binária do argumento "n"
        Q = self.copy()
        A = self.zero()
        for b in m:
            if b == 1:
                A.soma(Q)
            Q.duplica()
        return A



In [102]:
class edCurve(object):
    def __init__(self,p, a, d , ed = None):
        assert a != d and is_prime(p) and p > 3
        K= GF(p) 
  
        A =  2*(a + d)/(a - d)
        B =  4/(a - d)
    
        alfa = A/(3*B) ; s = B

        a4 =  s**(-2) - 3*alfa**2
        a6 =  -alfa**3 - a4*alfa
        
        self.K = K
        self.constants = {'a': a , 'd': d , 'A':A , 'B':B , 'alfa':alfa , 's':s , 'a4':a4 , 'a6':a6 }
        self.EC = EllipticCurve(K,[a4,a6]) 
        
        if ed != None:
            self.L = ed['L']
            print(ed['L'])
            self.P = self.ed2ec(ed['Px'],ed['Py'])  # gerador do gru
        else:
            self.gen()
    
    def order(self):
        # A ordem prima "n" do maior subgrupo da curva, e o respetivo cofator "h" 
        oo = self.EC.order()
        n,_ = list(factor(oo))[-1]
        return (n,oo//n)
    
    def gen(self):
        L, h = self.order()       
        P = O = self.EC(0)
        while L*P == O:
            P = self.EC.random_element()
        self.P = h*P ; self.L = L

  
    
    def is_edwards(self, x, y):
        a = self.constants['a'] ; d = self.constants['d']
        x2 = x**2 ; y2 = y**2
        return a*x2 + y2 == 1 + d*x2*y2

    def ed2ec(self,x,y):      ## mapeia edCurve --> EC
        if (x,y) == (0,1):
            return self.EC(0)
        z = (1+y)/(1-y) ; w = z/x
        alfa = self.constants['alfa']; s = self.constants['s']
        return self.EC(z/s + alfa , w/s)
    
    def ec2ed(self,P):        ## mapeia EC --> edCurve
        if P == self.EC(0):
            return (0,1)
        x,y = P.xy()
        alfa = self.constants['alfa']; s = self.constants['s']
        u = s*(x - alfa) ; v = s*y
        return (u/v , (u-1)/(u+1))
    

In [103]:

class EdDSA:
    storage = []
    
    def __init__(self, ed):
        if(ed=='ed25519'):
            print('Escolhida a curva Ed25519.')
            self.setup_ed25519()
        else:
            print('Escolhida a curva Ed448.')
            self.setup_ed448()

    def setup_ed25519(self):
        p = 2**255-19
        K = GF(p)
        a = K(-1)

        d = -K(121665)/K(121666)
        #

        ed25519 = {
        'b'  : 256,
        'Px' : K(15112221349535400772501151409588531511454012693041857206046113283949847762202),
        'Py' : K(46316835694926478169428394003475163141307993866256225615783033603165251855960),
        'L'  : ZZ(2**252 + 27742317777372353535851937790883648493), ## ordem do subgrupo primo
        'n'  : 254,
        'h'  : 8
        }

        Px = ed25519['Px']; Py = ed25519['Py']

        E = edCurve(p,a,d,ed=None)
        G = edPoint(curve=E,x=Px,y=Py)
        l = E.order()[0]

        self.b = ed25519['b']
        self.requested_security_strength = 128
        self.E = E
        self.G = G
        self.l = l
        self.algorithm = 'ed25519'


    def setup_ed448(self):
        p = 2**448 - 2**224 - 1
        K = GF(p)
        a = K(1)
        d = K(-39081)

        ed448= {
        'b'  : 456,     ## tamanho das assinaturas e das chaves públicas
        'Px' : K(224580040295924300187604334099896036246789641632564134246125461686950415467406032909029192869357953282578032075146446173674602635247710) ,
        'Py' : K(298819210078481492676017930443930673437544040154080242095928241372331506189835876003536878655418784733982303233503462500531545062832660) ,                                          
        'L'  : ZZ(2**446 - 13818066809895115352007386748515426880336692474882178609894547503885) ,
        'n'  : 447,     ## tamanho dos segredos: os dois primeiros bits são 0 e o último é 1.
        'h'  : 4        ## cofactor
        }

        Px = ed448['Px']; Py = ed448['Py']

        E = edCurve(p,a,d, ed=None)
        G = edPoint(curve=E,x=Px,y=Py)
        l = E.order()[0]

        self.b = ed448['b']
        self.requested_security_strength = 224
        self.E = E
        self.G = G
        self.l = l
        self.algorithm = 'ed448'

    # hash function for each curve ED2556 and ED448
    def hash(self,data):
        if self.algorithm == 'ed25519':
            return hashlib.sha512(data).digest()
        else:
            return hashlib.shake_256(data).digest(912//8)

    # private key digest
    def digest(self,d):
        h = self.hash(d)
        buffer = bytearray(h)
        return buffer

    def s_value(self,h):
        if self.algorithm == 'ed25519':
            return self.s_value_ed25519(h)
        else:
            return self.s_value_ed448(h)

    def s_value_ed25519(self,h):
        digest = int.from_bytes(h, 'little')
        bits = [int(digit) for digit in list(ZZ(digest).binary())]
        x = 512 - len(bits)
        while x != 0:
            bits = [0] + bits
            x = x-1

        bits[0] = bits[1] = bits[2] = 0
        bits[self.b-2] = 1
        bits[self.b-1] = 0

        bits = "".join(map(str, bits))

        s = int(bits[::-1], 2)
        return s

    def s_value_ed448(self,h):
        digest = int.from_bytes(h, 'little')
        bits = [int(digit) for digit in list(ZZ(digest).binary())]
        x = 512 - len(bits)
        while x != 0:
            bits = [0] + bits
            x = x-1

        bits[0] = bits[1] = 0
        bits[self.b-9] = 1
        for i in bits[self.b-8:self.b]:
            bits[i] = 0

        bits = "".join(map(str, bits))

        s = int(bits[::-1], 2)
        return s

    # point encoding
    def encoding(self,Q, n):
        x, y = Q.x, Q.y
        self.storage.insert(n,(x,y))
        return x
    
    # point decoding
    def decoding(self,n):
        Q = self.storage[n]
        return Q
    
    # KeyGen
    def keyGen(self):
        # private key
        d = os.urandom(self.b//8)
        # s value
        digest = self.digest(d) 
        if self.algorithm == 'ed25519':
            bytes_length = 32
        else:
            bytes_length = 57

        hdigest1 = digest[:bytes_length]
        s = self.s_value(hdigest1)
        
        # public key
        T = self.G.mult(s)

        # public key encoding
        Q = self.encoding(T,0)
        Q = int(Q).to_bytes(bytes_length, 'little')
        return d, Q

    # domain separation tag
    def dom4(self, f, context): 
        init_string = []
        context_octets = []
        
        for c in context:
            context_octets.append(format(ord(c), "08b"))
        context_octets = ''.join(context_octets)

        for c in "SigEd448":
            init_string.append(format(ord(c), "08b"))
        init_string = ''.join(init_string)

        bits_int = int(init_string + format(f, "08b") + format(len(context_octets), "08b") + context_octets, 2)
        byte_array = bits_int.to_bytes((bits_int.bit_length() + 7) // 8, 'little')
        
        return byte_array
        
    # Sign
    def sign(self,M,d,Q,context = ''):
        # private key hash
        digest = self.digest(d)

        if self.algorithm == 'ed25519':
            bytes_length = 32
            hashPK = digest[bytes_length:]
            hashPK_old = digest[:bytes_length]
            r = self.hash(hashPK+M)
        else:
            bytes_length = 57
            hashPK = digest[bytes_length:]
            hashPK_old = digest[:bytes_length]
            r = self.hash(self.dom4(0, context)+hashPK+M)
        
        # r value
        r = int.from_bytes(r, 'little')

        # calculate R and encoding it
        R = self.G.mult(r)
        Rx = self.encoding(R,1)
        R = int(Rx).to_bytes(bytes_length, 'little')

        # s value
        s = self.s_value(hashPK_old)
        
        if self.algorithm == 'ed25519':
            # (R || Q || M) hash
            hashString = self.hash(R+Q+M)
        else:
            # (dom4(0,context) || R || Q || M) hash
            hashString = self.hash(self.dom4(0, context)+R+Q+M)

        hashString = int.from_bytes(hashString, 'little')

        # S = (r + SHA-512(R || Q || M) * s) mod n
        S = mod(r + hashString * s,self.l)
        S = int(S).to_bytes(bytes_length, 'little')

        signature = R + S
        return signature
    
    # Verify
    def verify(self,M,A,Q, context = ''):
        if self.algorithm == 'ed25519':
            bytes_length = 32
        else:
            bytes_length = 57

        # get R and S from signature A
        R = A[:bytes_length]
        S = A[bytes_length:]
        s = int.from_bytes(S, 'little')

        # decoding S, R and Q
        if (s >= 0 and s < self.l):
            (Rx, Ry) = self.decoding(1)
            (Qx, Qy) = self.decoding(0)
            if(Rx != None and Qx != None):
                res = True
            else: return False
        else: return False

        # t value
        if self.algorithm == 'ed25519':
            digest = self.hash(R+Q+M)
        else:
            digest = self.hash(self.dom4(0, context)+R+Q+M)
            
        t = int.from_bytes(digest, 'little')

        # get variables for verifying process
        value = 2**3
        R = int.from_bytes(R, 'little')
        Q = int.from_bytes(Q, 'little')
        R = edPoint(curve=self.E,x=Rx,y=Ry)
        Q = edPoint(curve=self.E,x=Qx,y=Qy)

        # get verification conditions: [2**c * S]G == [2**c]R + (2**c * t)Q
        cond1 = self.G.mult(value*s)
        cond2 = R.mult(value)
        cond3 = Q.mult(value*t)
        cond2.soma(cond3)

        # final verification
        return cond1.eq(cond2)



In [104]:
"""
edDSA = EdDSA('ed448')
signed_message = "Esta mensagem está assinada!"
unsigned_message = "Esta mensagem não está assinada..."
print("Mensagem a ser assinada: " + signed_message)
privateKey, publicKey = edDSA.keyGen()
print("\nSK: ")
print(privateKey)
print("PK: ")
print(publicKey)
print()
assinatura = edDSA.sign(dumps(signed_message), privateKey, publicKey, 'contexto')
print("Assinatura: ")
print(assinatura)
print()
print("Verificação da autenticação da mensagem assinada:")
if edDSA.verify(dumps(signed_message), assinatura, publicKey, 'contexto')==True:
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada...")
    
print()
print("Verificação da autenticação da mensagem não assinada:")
if edDSA.verify(dumps(unsigned_message), assinatura, publicKey, 'contexto')==True:
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada...")
"""

'\nedDSA = EdDSA(\'ed448\')\nsigned_message = "Esta mensagem está assinada!"\nunsigned_message = "Esta mensagem não está assinada..."\nprint("Mensagem a ser assinada: " + signed_message)\nprivateKey, publicKey = edDSA.keyGen()\nprint("\nSK: ")\nprint(privateKey)\nprint("PK: ")\nprint(publicKey)\nprint()\nassinatura = edDSA.sign(dumps(signed_message), privateKey, publicKey, \'contexto\')\nprint("Assinatura: ")\nprint(assinatura)\nprint()\nprint("Verificação da autenticação da mensagem assinada:")\nif edDSA.verify(dumps(signed_message), assinatura, publicKey, \'contexto\')==True:\n    print("Mensagem autenticada!")\nelse:\n    print("Mensagem não autenticada...")\n    \nprint()\nprint("Verificação da autenticação da mensagem não assinada:")\nif edDSA.verify(dumps(unsigned_message), assinatura, publicKey, \'contexto\')==True:\n    print("Mensagem autenticada!")\nelse:\n    print("Mensagem não autenticada...")\n'

In [91]:
edDSA = EdDSA('ed25519')
signed_message = "Esta mensagem está assinada!"
unsigned_message = "Esta mensagem não está assinada..."
print("Mensagem a ser assinada: " + signed_message)
privateKey, publicKey = edDSA.keyGen()
print("\nSK: ")
print(privateKey)
print("PK: ")
print(publicKey)
print()
assinatura = edDSA.sign(dumps(signed_message), privateKey, publicKey)
print("Assinatura: ")
print(assinatura)
print()
print("Verificação da autenticação da mensagem assinada:")
if edDSA.verify(dumps(signed_message), assinatura, publicKey)==True:
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada...")
    
print()
print("Verificação da autenticação da mensagem não assinada:")
if edDSA.verify(dumps(unsigned_message), assinatura, publicKey)==True:
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada...")

Escolhida a curva Ed25519.
7237005577332262213973186563042994240857116359379907606001950938285454250989
Mensagem a ser assinada: Esta mensagem está assinada!

SK: 
b'T\x8d0+\xd7@\x99bj\xca\x88\x08nf\x86\xfb\x18<;\xd7^\xe5\xc0\x16\x19\xadn%\xe0\xb8\x8c\xae'
PK: 
b'\xd8\x014X\x08\x00^\x8e^[\xccUd\xf8\x981\xa0\x95\xebC\xb0\x88\xc6\xc4\x19\xd6\xc9\xdaJ\x1a\tp'

Assinatura: 
b"C\xb3\xcf\xab\xf8A\xdcM\xcb\xfd\xcfz\x99\xa9'\xd9\xf4j\xe3\x99k#\xcb\xe5Z\xac2\x0b\xe5\x1d\x84\x19\x8dd\x85\xab\x93\x8a\x8bX\x9e\n\xfc`c\xee&\xdeR\xb5\xba%\x94%\x9a&\xc8h\x18N\xa3C\x8e\x02"

Verificação da autenticação da mensagem assinada:
Mensagem autenticada!

Verificação da autenticação da mensagem não assinada:
Mensagem não autenticada...


# Sem SageMath

In [92]:
import hashlib;
import os;

In [93]:
def sqrt4k3(x,p): 
    return pow(x, (p + 1) // 4, p)


def sqrt8k5(x,p):
    y = pow(x, (p + 3) // 8, p)

    if (y * y) % p == x % p: 
        return y
    else:
        z = pow(2, (p - 1) // 4, p)
        return (y * z) % p


def from_le(s): 
    return int.from_bytes(s, byteorder="little")


def shake256(data, olen):
    hasher = hashlib.shake_256()
    hasher.update(data)

    return hasher.digest(olen)


def sha512(data):
    return hashlib.sha512(data).digest()


def ed448_hash(data):
    dompfx = b"SigEd448" + bytes([0, 0])

    return shake256(dompfx + data, 114)



In [94]:

class Field:

    def __init__(self, x, p):
        self.x = x % p
        self.p = p


    def check_fields(self, y):
        if type(y) is not Field or self.p != y.p:
            raise ValueError("[ERROR] fields don't match")    
    

    def __add__(self,y):
        self.check_fields(y)
        return Field(self.x + y.x, self.p)
    

    def __sub__(self, y):
        self.check_fields(y)
        return Field(self.p + self.x - y.x, self.p)
        

    def __neg__(self):
        return Field(self.p - self.x, self.p)
    

    def __mul__(self,y):
        self.check_fields(y)
        return Field(self.x * y.x, self.p)
    

    def __truediv__(self,y):
        return self * y.inv()
    

    def inv(self):
        return Field(pow(self.x, self.p - 2, self.p), self.p)
    

    def sqrt(self):
        if self.p % 4 == 3: 
            y = sqrt4k3(self.x, self.p)

        elif self.p % 8 == 5: 
            y = sqrt8k5(self.x, self.p)
        else: 
            raise NotImplementedError("[ERROR] sqrt")
        
        _y = Field(y, self.p)

        return _y if _y * _y == self else None
    

    def make(self, ival): 
        return Field(ival, self.p)
    
    
    def iszero(self): 
        return self.x == 0
    
    
    def __eq__(self,y): 
        return self.x == y.x and self.p == y.p

    
    def __ne__(self,y): 
        return not (self == y)
    
    
    def tobytes(self, b):
        return self.x.to_bytes(b // 8, byteorder="little")
    
    
    def frombytes(self, x, b):
        rv = from_le(x) % (2 ** (b - 1))
        return Field(rv, self.p) if rv < self.p else None
    
    
    def sign(self): 
        return self.x % 2



In [95]:

class EdwardsPoint:

    def initpoint(self, x, y):
        self.x = x
        self.y = y
        self.z = self.base_field.make(1)


    def decode_base(self, s, b):
        if len(s) != b // 8: 
            return (None, None)
        
        xs = s[(b - 1) // 8] >> ((b - 1) & 7)

        y = self.base_field.frombytes(s, b)
        if y is None: 
            return (None, None)

        x = self.solve_x2(y).sqrt()
        if x is None or (x.iszero() and xs != x.sign()):
            return (None, None)

        if x.sign() != xs: 
            x = -x

        return (x, y)
    

    def encode_base(self, b):
        xp, yp = self.x / self.z, self.y / self.z

        s = bytearray(yp.tobytes(b))

        if xp.sign() != 0: 
            s[(b - 1) // 8] |= 1 << (b - 1) % 8

        return s
    

    def __mul__(self, x):
        r = self.zero_elem()
        s = self

        while x > 0:
            if (x % 2) > 0:
                r = r + s
            s = s.double()
            x = x // 2

        return r
    

    def __eq__(self, y):
        xn1 = self.x * y.z
        xn2 = y.x * self.z
        yn1 = self.y * y.z
        yn2 = y.y * self.z

        return xn1==xn2 and yn1==yn2
    

    def __ne__(self,y): 
        return not (self == y)



In [96]:

class Edwards25519Point(EdwardsPoint):

    base_field = Field(1, 2 ** 255 - 19)
    d = -base_field.make(121665) / base_field.make(121666)
    f0 = base_field.make(0)
    f1 = base_field.make(1)
    xb = base_field.make(15112221349535400772501151409588531511454012693041857206046113283949847762202)
    yb = base_field.make(46316835694926478169428394003475163141307993866256225615783033603165251855960)


    @staticmethod
    def stdbase():
        return Edwards25519Point(Edwards25519Point.xb, Edwards25519Point.yb)
    

    def __init__(self, x, y):
        if y * y - x * x != self.f1 + self.d * x * x * y * y:
            raise ValueError("[ERROR] invalid point")
        self.initpoint(x, y)
        self.t = x * y


    def decode(self, s):
        x, y = self.decode_base(s, 256)
        return Edwards25519Point(x, y) if x is not None else None


    def encode(self):
        return self.encode_base(256)
    

    def zero_elem(self):
        return Edwards25519Point(self.f0, self.f1)
    

    def solve_x2(self, y):
        return ((y * y - self.f1) / (self.d * y * y + self.f1))
    

    def __add__(self, y):
        tmp = self.zero_elem()
        zcp = self.z * y.z
        A = (self.y - self.x) * (y.y - y.x)
        B = (self.y + self.x) * (y.y + y.x)
        C = (self.d + self.d) * self.t * y.t
        D = zcp + zcp
        E, H = B - A, B + A
        F, G = D - C, D + C
        tmp.x, tmp.y, tmp.z, tmp.t = E * F, G * H, F * G, E * H

        return tmp
    

    def double(self):
        tmp = self.zero_elem()
        A = self.x * self.x
        B = self.y * self.y
        Ch = self.z * self.z
        C = Ch + Ch
        H = A + B
        xys = self.x + self.y
        E = H - xys * xys
        G = A - B
        F = C + G
        tmp.x, tmp.y, tmp.z, tmp.t = E * F, G * H, F * G, E * H

        return tmp
    

    def l(self):
        return 7237005577332262213973186563042994240857116359379907606001950938285454250989
    

    def c(self): 
        return 3


    def n(self): 
        return 254


    def b(self): 
        return 256


    def is_valid_point(self):
        x, y, z, t = self.x, self.y, self.z, self.t
        x2 = x * x
        y2 = y * y
        z2 = z * z
        lhs = (y2 - x2) * z2
        rhs = z2 * z2 + self.d * x2 * y2
        assert(lhs == rhs)
        assert(t * z == x * y)
        

In [97]:

class Edwards448Point(EdwardsPoint):

    base_field = Field(1, 2 ** 448 - 2 ** 224 - 1)
    d = base_field.make(-39081)
    f0 = base_field.make(0)
    f1 = base_field.make(1)
    xb = base_field.make(224580040295924300187604334099896036246789641632564134246125461686950415467406032909029192869357953282578032075146446173674602635247710)
    yb = base_field.make(298819210078481492676017930443930673437544040154080242095928241372331506189835876003536878655418784733982303233503462500531545062832660)


    @staticmethod
    def stdbase():
        return Edwards448Point(Edwards448Point.xb, Edwards448Point.yb)
    

    def __init__(self, x, y):
        if y * y + x * x != self.f1 + self.d * x * x * y * y:
            raise ValueError("[ERROR] invalid point")
        self.initpoint(x, y)


    def decode(self, s):
        x, y = self.decode_base(s, 456)
        return Edwards448Point(x, y) if x is not None else None
    

    def encode(self):
        return self.encode_base(456)
    

    def zero_elem(self):
        return Edwards448Point(self.f0, self.f1)
    

    def solve_x2(self,y):
        return ((y*y-self.f1)/(self.d*y*y-self.f1))


    def __add__(self,y):
        tmp = self.zero_elem()
        xcp, ycp, zcp = self.x * y.x, self.y * y.y, self.z * y.z
        B = zcp * zcp
        E = self.d * xcp * ycp
        F, G = B - E, B + E
        tmp.x = zcp * F *((self.x + self.y) * (y.x + y.y) - xcp - ycp)
        tmp.y, tmp.z = zcp * G * (ycp - xcp), F * G

        return tmp
    

    def double(self):
        tmp = self.zero_elem()
        x1s, y1s, z1s = self.x * self.x, self.y * self.y, self.z * self.z
        xys = self.x + self.y
        F = x1s + y1s
        J = F - (z1s + z1s)
        tmp.x, tmp.y, tmp.z = (xys * xys - x1s - y1s) * J, F * (x1s - y1s), F * J

        return tmp
    

    def l(self):
        return 181709681073901722637330951972001133588410340171829515070372549795146003961539585716195755291692375963310293709091662304773755859649779


    def c(self): 
        return 2
    

    def n(self): 
        return 447
    

    def b(self): 
        return 456
    

    def is_valid_point(self):
        x, y, z = self.x, self.y, self.z
        x2 = x * x
        y2 = y * y
        z2 = z * z
        lhs = (x2 + y2) * z2
        rhs = z2 * z2 + self.d * x2 * y2
        assert(lhs == rhs)



In [98]:

class EdDSA:

    def __init__(self, curve):
        if curve == 'edwards25519':
            self.B = Edwards25519Point.stdbase()
            self.H = sha512
        
        elif curve == 'edwards448':
            self.B = Edwards448Point.stdbase()
            self.H = ed448_hash

        else:
            raise ValueError("[ERROR] not accepted curve name")

        self.l = self.B.l()
        self.n = self.B.n()
        self.b = self.B.b()
        self.c = self.B.c()


    def clamp(self, a):
        _a = bytearray(a)
        for i in range(0, self.c): 
            _a[i // 8] &= ~(1 << (i % 8))
        _a[self.n // 8] |= 1 << (self.n % 8)

        for i in range(self.n + 1, self.b): 
            _a[i // 8] &= ~(1 << (i % 8))

        return _a
    

    def keygen(self, privkey):
        if privkey is None: 
            privkey = os.urandom(self.b // 8)

        khash = self.H(privkey)
        a = from_le(self.clamp(khash[:self.b // 8]))

        return privkey, (self.B * a).encode()
    

    def sign(self, privkey, pubkey, msg):
        khash = self.H(privkey)
        a = from_le(self.clamp(khash[:self.b // 8]))
        seed = khash[self.b // 8:]
        r = from_le(self.H(seed + msg)) % self.l
        R = (self.B * r).encode()
        h = from_le(self.H(R + pubkey + msg)) % self.l
        S = ((r + h * a) % self.l).to_bytes(self.b // 8, byteorder="little")

        return R + S
    

    def verify(self, pubkey, msg, sig):
        if len(sig) != self.b // 4: 
            return False
        
        if len(pubkey) != self.b // 8: 
            return False
        
        Rraw , Sraw = sig[:self.b // 8], sig[self.b // 8:]
        R, S = self.B.decode(Rraw), from_le(Sraw)
        A = self.B.decode(pubkey)

        if (R is None) or (A is None) or S >= self.l: 
            return False
        
        h = from_le(self.H(Rraw + pubkey + msg)) % self.l

        rhs = R + (A * h)
        lhs = self.B * S
        for i in range(0, self.c):
            lhs = lhs.double()
            rhs = rhs.double()

        return lhs == rhs



In [99]:

ed25519 = EdDSA('edwards25519')
ed448 = EdDSA('edwards448')
priv, pub = ed448.keygen(None)
# print('priv:', priv)
# print('pub:', pub)

sign = ed448.sign(priv, pub, b'cripto')
# print('sign:', sign)

a = ed448.verify(pub, b'cripto', sign)
print(a)


priv, pub = ed25519.keygen(None)
# print('priv:', priv)
# print('pub:', pub)

sign = ed25519.sign(priv, pub, b'cripto')
# print('sign:', sign)

a = ed25519.verify(pub, b'cripto', sign)
print(a)

True
True
